In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import re  #正则表达式
import nltk 


## 读入数据 

In [10]:
data=pd.read_csv('youtube_comment.csv') #读入数据
data.info()  #查看数据信息
print(data['Class'].value_counts())
data.head(10) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Author  344 non-null    object
 1   Date    344 non-null    object
 2   Text    344 non-null    object
 3   Class   344 non-null    object
dtypes: object(4)
memory usage: 10.9+ KB
spam    172
ham     172
Name: Class, dtype: int64


,Author,Date,Text,Class
0,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",spam
1,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,spam
2,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,spam
3,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ?,spam
4,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .?,spam
5,Jason Haddad,2013-11-26T02:55:11,"Hey, check out my new website!! This site is a...",spam
6,ferleck ferles,2013-11-27T21:39:24,Subscribe to my channel ?,spam
7,Bob Kanowski,2013-11-28T12:33:27,i turned it on mute as soon is i came on i jus...,ham
8,Cony,2013-11-28T16:01:47,You should check my channel for Funny VIDEOS!!?,spam
9,BeBe Burkey,2013-11-28T16:30:13,and u should.d check my channel and tell me wh...,spam


## 词形还原 

In [11]:
from nltk.corpus import wordnet
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None



## 预处理

In [12]:
from nltk.corpus import stopwords
wnl = nltk.stem.WordNetLemmatizer()
data['Text_words']=''
for i in range(data.shape[0]):
    text=re.sub(r'[^\w\s]','',data.iloc[i][2])  #正则表达式，去除标点符号
    text=text.lower()  #小写
    tokenize = nltk.word_tokenize(text) #英文分词
    tokenize_letter=[w for w in tokenize if w.isalpha() ]  #删除数字
    tokenize_text = [w for w in tokenize_letter if not w in stopwords.words('english')] #去除停用词
    pos_tags=nltk.pos_tag(tokenize_text)#词性标注
    words_lemma=[]
    for pos in pos_tags:
        if get_wordnet_pos(pos[1])!=None : #根据上面函数，J,V,N,R开头的保留，其他返回空值，如果不为空，表示如果为JVNR
            wordnet_pos =get_wordnet_pos(pos[1]) #pos[1]为标注的词性
            words_lemma.append(wnl.lemmatize(pos[0], pos=wordnet_pos)) # 词形还原 
    data['Text_words'].iloc[i]=words_lemma
data.head(100)

,Author,Date,Text,Class,Text_words
0,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",spam,"[huh, anyway, check, youtube, channel]"
1,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,spam,"[hey, guy, check, new, channel, first, vid, mo..."
2,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,spam,"[test, say, murdevcom]"
3,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ?,spam,"[shake, sexy, as, channel, enjoy]"
4,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .?,spam,"[watchvvtarggvgtwq, check]"
...,...,...,...,...,...
95,Jack ToadROXMK,2014-11-04T02:00:14,Anybody who subscribes to me will get 10 subsc...,spam,"[anybody, subscribe, get, subscriber]"
96,roflcopter2110,2014-11-04T02:22:06,http://thepiratebay.se/torrent/6381501/Timothy...,spam,[]
97,Imprezzi Vidzz,2014-11-04T03:12:23,"My videos are half way decent, check them out ...",spam,"[video, way, decent, check, want]"
98,khir abqari,2014-11-04T07:37:28,they said this video are not deserve 2billion ...,ham,"[say, video, deserve, view, keep, visit, watch..."


In [13]:
# 准备语料库
corpus=[]
for i in range (data.shape[0]):
    corpus.append(re.sub(r'[^\w\s]','',str(data['Text_words'][i])))
print(corpus)

['huh anyway check youtube channel', 'hey guy check new channel first vid monkey im monkey white shirtplease leave comment please subscribe', 'test say murdevcom', 'shake sexy as channel enjoy', 'watchvvtarggvgtwq check', 'hey check new website site kid stuff kidsmediausa com', 'subscribe channel', 'turn mute soon come want check view', 'check channel funny video', 'u shouldd check channel tell', 'hey subscribe', 'start read stop subscribe day youre entire family die want stay alive subscribe right', 'httpstwittercomgbphotographygb', 'subscribe comment', 'please', 'hello game art video scientific experiment tutorial lyric videos much much please check channel subscribe weve start soon hope able cover expectation also check weve get far', 'im check view', '', '', 'edm apparel company dedicate bring music inspire design clothe perfect rave music festival neon crop top tank tops tshirts vnecks accessory follow facebook instagraml free giveaway news visit site oncueapparel', 'think million

In [16]:
#unigram
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 1)) #创建词袋子  1元 和2元，默认1元
dt = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names_out())
print(dt.toarray())  

['aaaaaaa' 'able' 'absolutely' 'access' 'accessory' 'account' 'act'
 'active' 'actor' 'actually' 'add' 'admit' 'adsense' 'advice' 'ago' 'al'
 'alive' 'allot' 'allways' 'almost' 'alone' 'alot' 'also' 'amaze' 'amazon'
 'america' 'amount' 'amp' 'ana' 'android' 'angel' 'angry' 'animation'
 'annoy' 'anybody' 'anymore' 'anyone' 'anyway' 'apocalypse' 'app'
 'apparel' 'apparently' 'appreciate' 'apps' 'art' 'as' 'asia' 'asian'
 'attention' 'auburn' 'audition' 'avoid' 'away' 'aways' 'awesome'
 'awesomeness' 'baby' 'back' 'bad' 'band' 'bass' 'beautiful' 'behold'
 'beibs' 'believe' 'best' 'bieber' 'big' 'bighit' 'bilion' 'billion'
 'billioncoming' 'billon' 'bing' 'bit' 'bitch' 'block' 'blue' 'bomb'
 'book' 'bot' 'bother' 'bottom' 'boy' 'boyfriend' 'brain' 'brand' 'bring'
 'brother' 'bucket' 'burn' 'buy' 'call' 'camera' 'cant' 'capitalize' 'car'
 'card' 'cardsbut' 'care' 'caroline' 'cash' 'certain' 'chacking'
 'chainise' 'challenge' 'chance' 'chanel' 'change' 'chanicka' 'channel'
 'check' 'checked'

In [18]:
a=pd.DataFrame(dt.toarray())
a.columns=vectorizer.get_feature_names_out()
X=a
y=data['Class']

In [19]:
#分类模型
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection  import cross_val_score
knn = KNeighborsClassifier(n_neighbors=3)
scores = cross_val_score(knn,X,y, cv=5)#5折交叉验证
scores.mean()#平均准确率

0.6774083546462064